In [6]:
import pathlib

In [14]:
#import splitfolders

#path1 = "/Users/laurameyer/code/DSP-Tan/mushroom_learning/raw_data/1_12_mushroom_species"
#path2 = "/Users/laurameyer/code/DSP-Tan/mushroom_learning/raw_data/2_12_mushroom_species"
#path3 =  "/Users/laurameyer/code/DSP-Tan/mushroom_learning/raw_data/3_12_mushroom_species"

#splitfolders.ratio(path1, output="../raw_data/1_12_mushroom_species_train_test",
#                   seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

#splitfolders.ratio(path2, output="../raw_data/2_12_mushroom_species_train_test",
#                   seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

#splitfolders.ratio(path3, output="../raw_data/3_12_mushroom_species_train_test",
#                   seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

In [23]:
directory1 = "/Users/laurameyer/code/DSP-Tan/mushroom_learning/raw_data/1_12_mushroom_species_train_test/train"

In [24]:
def get_images_directory(directory):
    data_dir = pathlib.Path(directory)
    return data_dir 

data_dir = get_images_directory(directory1)

image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

4692